<a href="https://colab.research.google.com/github/Gunthee/Binary_tree/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_W03_PromptEngineering%5BTyphoon%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool setup

In [10]:
from google.colab import userdata

In [11]:
from google.colab import userdata
userdata.get('Typhoon') #adjust

'sk-wgVSEEMb7hEpZtSVksdDoVpIrPShhQRVWkbLhwr9yqRL2mhM'

In [12]:
import requests

In [25]:
messages = [
	{
		"role": "user",
		"content": "พูดไทยได้ไม๊"
	},
]

In [26]:
def generate_response(messages):
	endpoint = 'https://api.opentyphoon.ai/v1/chat/completions'
	res = requests.post(endpoint, json={
			"model": "typhoon-v2-8b-instruct",
			"max_tokens": 512,
			"messages": messages,
			"temperature": 0.1,
			"top_p": 0.5,
			"top_k": 0,
			"repetition_penalty": 1.05,
			"min_p": 0
	}, headers={"Authorization": f"Bearer {userdata.get('Typhoon')}",}) #Adjust
	return res.json()['choices'][0]['message']['content']

In [27]:
generate_response(messages)

'ใช่ครับ ผมสามารถพูดภาษาไทยได้'

# Prompt engineering

### Zero-shot

In [30]:
prompt = """Classify the sentiment of the following student feedback as positive, negative, or neutral.
Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
 Sentiment:"""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]

generate_response(messages)

'Positive'

### Few-shot

In [31]:
prompt = """Classify the sentiment of the following student feedback as positive, negative, or neutral.
Student feedback: ```ไม่มี```
Sentiment: neutral

Student feedback: ```เรียนสนุกดีครับ```
Sentiment: positive

Student feedback: ```อาจารย์ค่อนข้างใช้คำพูดที่แรงกับนักศึกษา```
Sentiment: negative

Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Sentiment:"""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]

generate_response(messages)

'neutral'

### [Zero] CoT

In [32]:
# 1st prompt reasoning

prompt = """Classify the sentiment of the following student feedback as positive, negative, or neutral. Response only onec sentiment that is mostly represented student sentiment.

Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Let's think step by step."""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]

prompt_1st = generate_response(messages)
print(prompt_1st)

1. Identify the key components of the feedback: "เรียนสนุกดี" and "อาจารย์สอนค่อนข้างเร็ว".
2. Analyze each component separately:
   - "เรียนสนุกดี": This part of the feedback indicates a positive sentiment.
   - "อาจารย์สอนค่อนข้างเร็ว": This part of the feedback suggests a negative sentiment.
3. Determine the overall sentiment: Since the positive sentiment ("เรียนสนุกดี") is more prominent than the negative sentiment ("อาจารย์สอนค่อนข้างเร็ว"), the overall sentiment of the feedback is positive.


In [33]:
# 2nd answer extraction

prompt = f"""Classify the sentiment of the following student feedback as positive, negative, or neutral. Response only onec sentiment that is mostly represented student sentiment.

Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Let's think step by step.

{prompt_1st}
There is a sentiment of:
"""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]

generate_response(messages)

'positive'

### Re2

In [34]:
# 1st prompt reasoning

prompt = """Classify the sentiment of the following student feedback as positive, negative, or neutral. Response only onec sentiment that is mostly represented student sentiment.

Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Read the feedback again: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Sentiment: Let's think step by step. Return the answer at the end of the response after a separator ####."""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]

prompt_1st = generate_response(messages)
print(prompt_1st)

The student feedback mentions that "เรียนสนุกดี" which indicates a positive sentiment about the learning experience being enjoyable. However, the phrase "อาจารย์สอนค่อนข้างเร็ว" suggests a slight negative sentiment regarding the pace of the teaching. Overall, the positive sentiment of enjoying the class seems to outweigh the negative sentiment about the teaching speed. 

####

Sentiment: Positive


### Tree-of-Thought

In [35]:
prompt = """Imagine three different experts are answering
this question. All experts will write down 1 step of their thinking, then share
it with the group. Then all experts will go on to the next step, etc. If any
expert realizes they're wrong at any point then they leave.

Classify the sentiment of the following student feedback as positive, negative, or neutral. Response only onec sentiment that is mostly represented student sentiment.

Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Read the feedback again: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Sentiment: Let's think step by step. Return the answer at the end of the response after a separator ####."""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]


print(generate_response(messages))

Expert 1: The student mentions that the class is enjoyable, which indicates a positive sentiment towards the learning experience.

Expert 2: However, the student also notes that the teacher teaches quite quickly. This could suggest that the pace of the lessons is not ideal for some students.

Expert 3: Overall, the feedback seems to be more positive than negative, as the student appreciates the enjoyable aspect of the class.

####

Sentiment: Positive


### Chain-of-Draft

In [36]:
prompt = """Classify the sentiment of the following student feedback as positive, negative, or neutral. Response only onec sentiment that is mostly represented student sentiment.

Student feedback: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Read the feedback again: ```เรียนสนุกดี แต่อาจารย์สอนค่อนข้างเร็ว```
Sentiment: Let's think step by step, but only keep a minimum draft for each thinking step, with 5 words at most. Return the answer at the end of the response after a separator ####"""

messages = [
    {
      "role": "user",
      "content": prompt
    },
]

generate_response(messages)

'Positive sentiment is mostly represented.  \n####  \nSentiment: Positive'

# LLM Application

### Summarization

In [37]:
review = "Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to her."


In [38]:
prompt =  """Summarize the review below, delimited by triple backticks, in at most 10 words.
Review: ```{paragraph}```""".format(paragraph=review)

print(prompt)

Summarize the review below, delimited by triple backticks, in at most 10 words.
Review: ```Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to her.```


In [39]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Your task is to generate a short summary of a product review from an ecommerce site."
    },
    {
      "role": "user",
      "content": prompt
    },
]

generate_response(messages)

'```markdown\nCute panda plush toy, but a bit small for the price.\n```'

### Classification

In [40]:
system_prompt = """You are a helpful assistant. Your task is to classify sentiment of a product review from an ecommerce site, which is delimited with triple backticks?
Give your answer as a single word, either "positive" or "negative".
"""

In [41]:
review = """I own this mouse in both colors now and use it everyday for both gaming and schoolwork.
After using my black one for about 1 1/2 years, I can say I've had no issues.
The light up detail on the mouse is pretty and matches my new setup- it's white on the white mouse and green on the black mouse.
The two side buttons are useful as key binds in many games and to go back and forth with webpages and I'm happy there isn't a DPI button near the scroll wheel- no accidental DPI changes.
The mouse also has a slightly matte texture on the top which makes using it enjoyable. Not a very heavy mouse, is a good size for an average length hand.
"""

In [42]:
prompt =  """Review: ```{paragraph}```""".format(paragraph=review)

print(prompt)

Review: ```I own this mouse in both colors now and use it everyday for both gaming and schoolwork.
After using my black one for about 1 1/2 years, I can say I've had no issues.
The light up detail on the mouse is pretty and matches my new setup- it's white on the white mouse and green on the black mouse.
The two side buttons are useful as key binds in many games and to go back and forth with webpages and I'm happy there isn't a DPI button near the scroll wheel- no accidental DPI changes.
The mouse also has a slightly matte texture on the top which makes using it enjoyable. Not a very heavy mouse, is a good size for an average length hand.
```


In [43]:
messages = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
      "role": "user",
      "content": prompt
    },
]

generate_response(messages)

'positive'

### Topic modeling

In [44]:
system_prompt = """You are a helpful assistant. Your task is to classify the topic of a article from news site, which is delimited with triple backticks?

Give your answer as list with 0 or 1 for each topic.

List of topics: nasa, local government, engineering, employee satisfaction, federal government
"""

In [45]:
article = """In a recent survey conducted by the government, public sector employees were asked to rate their level of satisfaction with the department they work at. The results revealed that NASA was the most popular department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, stating, "I'm not surprised that NASA came out on top. It's a great place to work with amazing people and incredible opportunities. I'm proud to be a part of such an innovative organization."

The results were also welcomed by NASA's management team, with Director Tom Johnson stating, "We are thrilled to hear that our employees are satisfied with their work at NASA. We have a talented and dedicated team who work tirelessly to achieve our goals, and it's fantastic to see that their hard work is paying off."

The survey also revealed that the Social Security Administration had the lowest satisfaction rating, with only 45% of employees indicating they were satisfied with their job. The government has pledged to address the concerns raised by employees in the survey and work towards improving job satisfaction across all departments.
"""

In [46]:
prompt =  """Article: ```{paragraph}```""".format(paragraph=article)

print(prompt)

Article: ```In a recent survey conducted by the government, public sector employees were asked to rate their level of satisfaction with the department they work at. The results revealed that NASA was the most popular department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, stating, "I'm not surprised that NASA came out on top. It's a great place to work with amazing people and incredible opportunities. I'm proud to be a part of such an innovative organization."

The results were also welcomed by NASA's management team, with Director Tom Johnson stating, "We are thrilled to hear that our employees are satisfied with their work at NASA. We have a talented and dedicated team who work tirelessly to achieve our goals, and it's fantastic to see that their hard work is paying off."

The survey also revealed that the Social Security Administration had the lowest satisfaction rating, with only 45% of employees indicating they were satisfied with t

In [47]:
messages = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
      "role": "user",
      "content": prompt
    },
]

print(generate_response(messages))

[1, 0, 0, 0, 1]
